In [1]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
key_credential = os.environ["AZURE_SEARCH_ADMIN_KEY"] if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else None
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_API_KEY"] if len(os.environ["AZURE_OPENAI_API_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

credential = key_credential or DefaultAzureCredential()

In [2]:
from langchain_openai import AzureOpenAIEmbeddings
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from langchain.vectorstores.azuresearch import AzureSearch

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

# Use API key if provided, otherwise use RBAC authentication
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_openai_embedding_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)   

vector_store = AzureSearch(
    azure_search_endpoint=endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="contoso-semantic-config"
) 

In [3]:
from langsmith import Client
client = Client()

os.environ['LANGCHAIN_PROJECT'] = "contoso-coffee-extraction"

In [4]:
from langchain_community.chat_message_histories.cosmos_db import CosmosDBChatMessageHistory

cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id="abc123",
        user_id="shivac"
    )
cosmos.prepare_cosmos()
cosmos.messages


[HumanMessage(content='What french options do you have?'),
 AIMessage(content="We have a few French-inspired items on our menu:\n\n1. Pain au Chocolat: A classic French pastry filled with chocolate. It's priced at $3.5.\n2. Croissant: A buttery, flaky pastry, freshly baked to golden perfection. It's priced at $2.0.\n3. Cafe au Lait: A coffee drink made with strong hot coffee and scalded milk. It's priced at $3.5.\n\nPlease let me know if you'd like more information about these items or if you're ready to place an order."),
 HumanMessage(content='can I have 2 of each?'),
 AIMessage(content="Absolutely, I've added these to your order:\n\n1. 2x Pain au Chocolat ($3.5 each)\n2. 2x Croissant ($2.0 each)\n3. 2x Cafe au Lait ($3.5 each)\n\nIs there anything else you would like to add to your order?"),
 HumanMessage(content='can you make my coffees hot?'),
 AIMessage(content="Definitely! We'll ensure that your Cafe au Lait drinks are served hot. I've made a note of this in your order. Is there

In [5]:
def cosmos_messages_to_string(messages: list) -> str:
    """
    Converts a list of messages to a string.
    """
    chat_str = ""
    for message in cosmos.messages:
        if message.type == "human":
            chat_str += "Customer: " + message.content + "\n"
        if message.type == "ai":
            chat_str += "Cafe Bot: " + message.content + "\n"
    return chat_str

chat_str = cosmos_messages_to_string(cosmos.messages)
print(chat_str)

Customer: What french options do you have?
Cafe Bot: We have a few French-inspired items on our menu:

1. Pain au Chocolat: A classic French pastry filled with chocolate. It's priced at $3.5.
2. Croissant: A buttery, flaky pastry, freshly baked to golden perfection. It's priced at $2.0.
3. Cafe au Lait: A coffee drink made with strong hot coffee and scalded milk. It's priced at $3.5.

Please let me know if you'd like more information about these items or if you're ready to place an order.
Customer: can I have 2 of each?
Cafe Bot: Absolutely, I've added these to your order:

1. 2x Pain au Chocolat ($3.5 each)
2. 2x Croissant ($2.0 each)
3. 2x Cafe au Lait ($3.5 each)

Is there anything else you would like to add to your order?
Customer: can you make my coffees hot?
Cafe Bot: Definitely! We'll ensure that your Cafe au Lait drinks are served hot. I've made a note of this in your order. Is there anything else you would like to specify or add to your order?
Customer: Thanks, my name is shiv

In [6]:
from langchain_openai.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment="gpt-4",
    streaming=True,
)

In [7]:
from typing import List, Dict, Optional
import json
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

class OrderedItem(BaseModel):
    """
    Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.
    """
    item: str = Field(description="Name of the item ordered")
    quantity: int = Field(description="Quantity of that ordered item")
    price: float = Field(description="Price of that ordered item")
    item_notes: Optional[str] = Field(description="Any additional notes about that item provided by the customer")
    
class OrderInfo(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract order details such item name, quantity, price, and any additional notes for each item.
    """
    order: List[OrderedItem] = Field(description="List of items ordered and their details")

order_schema = convert_to_openai_function(OrderInfo)
print(json.dumps(order_schema, indent=4))

{
    "name": "OrderInfo",
    "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract order details such item name, quantity, price, and any additional notes for each item.",
    "parameters": {
        "type": "object",
        "properties": {
            "order": {
                "description": "List of items ordered and their details",
                "type": "array",
                "items": {
                    "description": "Information about each ordered item such as item name, quantity, price and additional notes provided by the customer.",
                    "type": "object",
                    "properties": {
                        "item": {
                            "description": "Name of the item ordered",
                            "type": "string"
                        },
                        "quantity": {
                            "description": "Quantity of that o

In [8]:
class CustomerName(BaseModel):
    """
    Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.
    Extract the customer's name.
    """
    customer_name: str = Field(description="Name of the customer")
    
customer_name_schema = convert_to_openai_function(CustomerName)
print(json.dumps(customer_name_schema, indent=4))

{
    "name": "CustomerName",
    "description": "Input is a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant.\nExtract the customer's name.",
    "parameters": {
        "type": "object",
        "properties": {
            "customer_name": {
                "description": "Name of the customer",
                "type": "string"
            }
        },
        "required": [
            "customer_name"
        ]
    }
}


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

extraction_functions = [order_schema]

order_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract and save the pertinent details, specifically the ordered food items, quantity ordered, their prices and any additional notes requested for each ordered item included in the order. 
            Make sure to extract only those items that were ordered.
        """),
    ("user", "{input}")
])

extraction_model = llm.bind(
    functions=extraction_functions,
    function_call={"name": "OrderInfo"}
)

order_extraction_chain = order_prompt | extraction_model | JsonOutputFunctionsParser()
order_result = order_extraction_chain.invoke({"input": chat_str})
print(json.dumps(order_result, indent=4))


{
    "order": [
        {
            "item": "Pain au Chocolat",
            "quantity": 2,
            "price": 3.5,
            "item_notes": ""
        },
        {
            "item": "Croissant",
            "quantity": 2,
            "price": 2.0,
            "item_notes": ""
        },
        {
            "item": "Cafe au Lait",
            "quantity": 2,
            "price": 3.5,
            "item_notes": "Serve hot"
        }
    ]
}


In [10]:
extraction_functions = [customer_name_schema]

name_prompt = ChatPromptTemplate.from_messages([
    ("system", """
            You are a world class algorithm for extracting information in structured formats from a restaurant order conversation between a customer and an AI assistant.
            Your task is to extract the customer's name included in the order.
        """),
    ("user", "{input}")
])

extraction_model = llm.bind(
    functions=extraction_functions,
    function_call={"name": "CustomerName"}
)

name_extraction_chain = name_prompt | extraction_model | JsonOutputFunctionsParser()
name = name_extraction_chain.invoke({"input": chat_str})
customer_name = name["customer_name"]
print(f"Customer name: {customer_name}")

Customer name: shiva chittamuru


In [14]:
import pandas as pd

order_df = pd.DataFrame(order_result["order"])
order_df['item'] = order_df['item'].str.strip().str.lower()
order_df

,item,quantity,price,item_notes
0,pain au chocolat,2,3.5,
1,croissant,2,2.0,
2,cafe au lait,2,3.5,Serve hot


In [18]:
import pandas as pd  
from fuzzywuzzy import process  

menu_data = pd.read_json('../data/contoso.json') 
  
# Convert the 'Item_Name' column to lower case for standardization  
menu_data['item'] = menu_data['item'].str.lower()  
  
# Create a dictionary to store the item_ids  
item_ids = dict(zip(menu_data['item'], menu_data['id']))  
  
# Extract the item_ids for the item_names in final_order_json_list  
for item in order_result["order"]:  
    item_name = item['item'].lower()  
  
    # Use fuzzy matching to find the closest matching item in the menu  
    matched_item, _ = process.extractOne(item_name, item_ids.keys())  
  
    id = item_ids.get(matched_item)  
    item['id'] = id  
  
# Print the updated final_order_json_list  
for item in order_result["order"]:  
    print(item) 

{'item': 'Pain au Chocolat', 'quantity': 2, 'price': 3.5, 'item_notes': '', 'id': 'b009'}
{'item': 'Croissant', 'quantity': 2, 'price': 2.0, 'item_notes': '', 'id': 'b002'}
{'item': 'Cafe au Lait', 'quantity': 2, 'price': 3.5, 'item_notes': 'Serve hot', 'id': 'c015'}


In [19]:
def calculate_total(order_result):
    sum = 0
    for item in order_result:
        sum += item['price']*item['quantity']
    return sum

print("Your total is: {0}$".format(calculate_total(order_result['order'])))

Your total is: 18.0$


In [20]:
def print_receipt(order_result, customer_name, tax_percent=0.095):
    print("-----------Contoso Coffee Inc-----------\n")
    for item in order_result:
        print("{0} x {1} = {2}$".format(item['item'], item['quantity'], round(item['price']*item['quantity'], 2)))
        if item['item_notes'] != "":
            print("\t Notes: {0}".format(item['item_notes']))
    
    subtotal = calculate_total(order_result)
    tax = round(tax_percent*subtotal , 2)
    total = round(subtotal + tax, 2)
    print("\nSubtotal: {0}$".format(subtotal))
    print("Tax: {0}$".format(tax))
    print("Total: {0}$".format(total))   
    
    print(f"\nThank you for dining with us, {customer_name.title()}!")
    
print_receipt(order_result['order'], customer_name)

-----------Contoso Coffee Inc-----------

Pain au Chocolat x 2 = 7.0$
Croissant x 2 = 4.0$
Cafe au Lait x 2 = 7.0$
	 Notes: Serve hot

Subtotal: 18.0$
Tax: 1.71$
Total: 19.71$

Thank you for dining with us, Shiva Chittamuru!
